This following cell contains code for a script that crawls through a Steam game's reviews. It is supposed to use Selenium 4 to automatically open a Microsoft Edge webpage that then runs through the review section, grabbing each seperate "card" that contains a review. The review is sectioned off, prepped to be added to its own row, and each seperate element of the review is prepped to be added as seperate cells in a row in a csv file. The CSV file will look like the following: 
(Example review) 


| SteamId | ProfileURL | ReviewText | Review | ReviewLength(Chars) | PlayHours | DatePosted | 
|:------|:------|:------|:------|:------|:------|:------|
| cadel13 | https://steamcommunity.com/id/cadel13/| Excellent flight game, feels so good to fly the classic star wars ships with all the lights and sound that accompany them. Highly recommend it | Recommended| 119| 14.3 | 14.3 hrs on record|

However, I got this code from someone else's work, which uses a deprecated version of Selenium (Selenium 3), and I use Selenium 4. I believe I've updated the functions to match Selenium 4, but I am getting error after error. The current one is: 

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-102-4f0e3dfab779> in <module>
     31 while running:
     32     # get cards on the page
---> 33     cards = find_element(By.CLASS_NAME , "apphub_Card")
     34 
     35 for card in cards[-20]:   # only the tail end are new cards

NameError: name 'find_element' is not defined
    
I imported selenium.webdriver.common.by By, and from what I understand it should work. 
    
There's probably a ton of things wrong with this code, so if it is completely nonsensical then I will just go back to scraping reviews normally, which just takes longer. I also have the option of downloading datasets that have already been acquired from Kaggle, such as this one https://www.kaggle.com/datasets/najzeko/steam-reviews-2021 (that one contains 21 million reviews, so I will probably try and find a smaller dataset, this is just to serve as an example).  

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime
from openpyxl import Workbook
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with

driver = webdriver.Edge()

game_id = 1222730

# get current position of y scrollbar
last_position = driver.execute_script("return window.pageYOffset;")
template = 'https://steamcommunity.com/app/{}/positivereviews/?browsefilter=mostrecent'
template_with_language = 'https://steamcommunity.com/app/{}/positivereviews/?browsefilter=mostrecent&filterLanguage=english'

url = template_with_language.format(game_id)

reviews = []
review_ids = set()
running = True

driver.maximize_window()
driver.get(url)

while running:
# get cards on the page
    cards = card.find_element(By.CLASS_NAME , 'apphub_Card')
# only the tail end are new cards
for card in cards[-20]:   

# gamer profile url
    profile_url = card.find_element(By.XPATH , './/div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')

# steam id
    steam_id = profile_url.split('/')[-2]
        
        # check to see if I've already collected this review
if steam_id in review_ids:
    continue
else: review_ids.add(steam_id)

        # username
user_name = card.find_element(By.XPATH , './/div[@class="apphub_friend_block"]/div/a[2]').text

        # language of the review
date_posted = card.find_element(By.XPATH , './/div[@class="apphub_CardTextContent"]/div').text
review_content = card.find_element(By.XPATH , './/div[@class="apphub_CardTextContent"]').text.replace(date_posted,'').strip()    
    

        # recommendation
thumb_text = card.find_element(By.XPATH , './/div[@class="reviewInfo"]/div[2]').text
thumb_text    

        # amount of play hours
play_hours = card.find_element( By.XPATH , './/div[@class="reviewInfo"]/div[3]').text
play_hours    

        # save review
review = (steam_id, profile_url, review_content, thumb_text, review_length, play_hours, date_posted)
reviews.append(review)    
        
    # attempt to scroll down thrice.. then break
scroll_attempt = 0
while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
        sleep(0.5)
        curr_position = driver.execute_script("return window.pageYOffset;")
        
        if curr_position == last_position: scroll_attempt += 1
        sleep(0.5)
            
        if curr_position >= 3:
                running = False
                break
        else:
            last_position = curr_position
            break  # continue scraping the results

# shutdown the web driver
driver.close()

today = datetime.today().strftime('%Y%m%d')   
with open(f'Steam_Reviews_{game_id}_{today}.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['SteamId', 'ProfileURL', 'ReviewText', 'Review', 'ReviewLength(Chars)', 'PlayHours', 'DatePosted'])
    writer.writerows(reviews)
    writer.close(reviews)